In [1]:
include("./dataManager.jl")

# Implementation of the Bottom Up Network with two Subversions: 
# BK with different Kernelsize and BF with different feature size
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehot, onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON
using LinearAlgebra

using MAT # needs installation of Pkg.add("MAT")
using PyPlot # pip install Matplotlib Pkg.add("PyPlot")
using ..dataManager: make_batch
using FeedbackConvNets
using Base

In [2]:
# PARAMETERS
# ----------
batch_size = 100
n = 5
c = 1
alpha = Float32(10^-4)
beta = 0.5f0
lambda = 0.0005f0

kernelsize = (3, 3)
featuresize = 32

momentum = 0.9f0
learningRate = 0.01f0
epochs = 100
decay_rate = 0.1f0
decay_step = 40

40

In [3]:
# Input is either 32x32x32xN or 16x16x32xN
function localResponseNorm(x)
    w = zeros(size(x))
    for k = 1:size(x,3)
        # calculate the boundaries of the sum
        # ATTENTION: this will sum n+1 Elements
        lower = Int32(max(1, floor(k-n/2)))
        upper = Int32(min(size(x,3), floor(k+n/2)))
        
        # square and sum 
        # w = map((x) -> x^2, x[:, :, lower:upper, :])
        norm = Tracker.data(x[:, :, lower:upper, :])
        norm = norm .^ 2
        # reduce to one 32x32x1xN or 16x16x1xN matrix
        norm = sum(norm, dims=3)
        norm = norm .* alpha
        norm = norm .+ c
        norm = norm .^ (-beta)
        w[:, :, k, :] = norm
    end   
    # a tracked array multiplied with a non tracked array returns a tracked array  
    x = w .* x
    return x
end

function loss(x, y)
    #y_hat = model(x)
    #cr = crossentropy(y_hat, y) 
    #sum2 = lambda * sum(norm, params(model))
    #display(params(model))
    #return cr + sum2
    y_hat = model(x)
    cr = crossentropy(y_hat, y) 
    sum2 = sum(norm, params(model))
    display(params(model))
    display(typeof(sum2))
    return cr + sum2
end

# loss = loss |> gpu

training = 0
function training_callback()
    training = training + 1
    @info("now at batch $(training)")
end

training_callback (generic function with 1 method)

In [4]:
@info("Constructing model...")
model = Chain(
    # HIDDEN LAYER 1
    # Input Image 32x32x1xN
    Conv(kernelsize, 1=>featuresize, pad=(1,1), relu),
    # local response normalization
    x -> localResponseNorm(x),
    MaxPool((2, 2), stride=(2, 2)),
    
    # HIDDEN LAYER 2
    # Input Image 16x16x32xN
    Conv(kernelsize, featuresize=>featuresize, pad=(1,1), relu),
    # local response normalization
    x -> localResponseNorm(x),
    MaxPool((16, 16), stride=(1, 1)),
    # reshape to 32xN
    x -> reshape(x, :, size(x, 4)),
    Dense(32, 10, σ),
)

# test the model (precomile it??)
#model = model |> gpu
model(rand(32, 32, 1, 1))

┌ Info: Constructing model...
└ @ Main In[4]:1


Tracked 10×1 Array{Float32,2}:
 0.54341644f0
 0.6481357f0 
 0.23100382f0
 0.33359793f0
 0.5924079f0 
 0.22047447f0
 0.7265448f0 
 0.79020816f0
 0.5094831f0 
 0.5608709f0 

In [5]:
# get the dataset
train_set, mean_img, std_img = make_batch(batch_size=10)

@info("Training...")
optimizer = Momentum(learningRate, momentum)
# using the momentum update rule

# test_set = gpu.(test_set)


train_set = gpu.(train_set)

label = zeros(10, 2)
label[3, 1] = 1
label[4, 2] = 1
data = [(rand(32,32,1,2), label), (rand(32,32,1,2), label)]
data = gpu.(data) # doesn't convert to float32!!

# display(train_set[1][1])
# display(mean_img)
# display(std_img)
# training loop

losst = loss(train_set[1][1], train_set[1][2])
#display(typeof(losst))

# Flux.train!(loss, params(model), train_set, optimizer, cb = training_callback)


┌ Info: Reading .mat file form source ../digitclutter/src/light_debris/light_debris_with_debris.mat
└ @ Main.dataManager /home/sebastian/GIT/NNFeedbackOperations/Src/dataManager.jl:39
┌ Info: calculate mean and standart deviation of dataset
└ @ Main.dataManager /home/sebastian/GIT/NNFeedbackOperations/Src/dataManager.jl:55
┌ Info: creating batches
└ @ Main.dataManager /home/sebastian/GIT/NNFeedbackOperations/Src/dataManager.jl:73
┌ Info: Training...
└ @ Main In[5]:4


Params([Float32[-0.341443 0.0265776 -0.369339; 0.382361 0.107868 0.0192445; 0.100131 -0.0275369 -0.300151]

Float32[0.231456 -0.050507 0.386814; 0.215755 0.0442183 0.00394429; 0.149338 0.00924802 0.331916]

Float32[0.287478 0.103428 0.0771892; 0.208912 0.340392 -0.0410792; 0.137974 -0.237877 0.30995]

...

Float32[0.0598139 -0.334501 -0.344253; 0.207081 0.226588 0.021283; 0.355205 0.0666841 0.316642]

Float32[-0.256002 0.0181124 0.134672; -0.127226 0.334112 0.226276; -0.0490724 -0.0804722 0.182184]

Float32[-0.00844192 0.368635 -0.300859; 0.116597 -0.155978 0.0875092; -0.143139 0.298988 -0.0474887] (tracked), Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] (tracked), Float32[0.170274 0.0916203 -0.161485; 0.121573 -0.241848 -0.122713; 0.0773857 -0.187573 0.052955]

Float32[-0.191078 -0.238403 -0.220442; -0.0350263 -0.13887 0.0326772; -0.116944 0.276322 0.261597]

Float32[0.197514 -0.291552 -0.239531; 0.076442 0.161019 0.0360

Tracker.TrackedReal{Float64}

25.23026352141214 (tracked)